# pytorch Tensor 操作总结
## Indexing & Slicing
### Basics
- x[a:b]：第0维切片

- x[:, a:b]：第1维切片

- x[..., a:b]：最后一维切片（前面维度全保留）

- x[:, -1]：取最后一个元素（降维）

- x[:, -1:]：取最后一个元素（保维）

**NOTE**: -1 会“降维”，-1: 会“保维”

### Bool Mask
- x[mask]：按 True 选元素（结果 shape 常不直观，容易变 1D）

### Advanced index: select / narrow / index_select
- torch.index_select, torch.masked_select, torch.where, torch.take

In [1]:
import torch
# generate 2d  5 x 5 random tensor
t = torch.rand(5, 5)

print(t[2:3, :])  # 3rd row
print(t[:, 2:3])  # 3rd column
print(t[:, -1:])  # last column
print(t[:, -1])  # last column, squeezed

print(t.select(dim=1, index=3) )         # 等价 t[:, 3]（会降维）
print(t.narrow(dim=1, start=2, length=2)) # 等价 t[:, 2:4]
idx = torch.tensor([1, 3, 4])  # 1D LongTensor
print(t.index_select(dim=1, index=idx))

tensor([[0.1487, 0.7390, 0.8095, 0.8403, 0.8885]])
tensor([[0.4682],
        [0.9542],
        [0.8095],
        [0.0101],
        [0.4852]])
tensor([[0.4790],
        [0.9332],
        [0.8885],
        [0.2712],
        [0.0032]])
tensor([0.4790, 0.9332, 0.8885, 0.2712, 0.0032])
tensor([0.5114, 0.6049, 0.8403, 0.9998, 0.3584])
tensor([[0.4682, 0.5114],
        [0.9542, 0.6049],
        [0.8095, 0.8403],
        [0.0101, 0.9998],
        [0.4852, 0.3584]])
tensor([[0.8249, 0.5114, 0.4790],
        [0.0872, 0.6049, 0.9332],
        [0.7390, 0.8403, 0.8885],
        [0.8078, 0.9998, 0.2712],
        [0.5394, 0.3584, 0.0032]])


## Shape / View ops（形状变换）
### reshape / view / flatten
- x.view(...)：不拷贝，要求 memory contiguous（不满足会报错）

- x.reshape(...)：尽量不拷贝，不行就拷贝（更稳）

- x.flatten(start_dim=1)：从某维起压平（常用在 [B,C,H,W]->[B,CHW]）

### unsqueeze / squeeze（加/删长度为1的维）

- x.unsqueeze(dim) 或 x[:, None]：插一个 1 维（为 broadcast/gather 做准备）

- x.squeeze(dim)：删掉 dim 上长度为1的维（dim 不写会删所有 1 维，容易坑）

**NOTE**: `squeeze` removes all dimensions that have a size of 1

### transpose / permute / movedim（换维度顺序）

- x.transpose(i,j)：交换两个维

- x.permute(...)：任意重排维度顺序

- x.movedim(src, dst)：把某个维挪到新位置（读起来更语义化）

**NOTE**: x = x.permute(...).contiguous().view(...) 因为 permute 往往让 tensor 非连续，要 .contiguous() 才能 .view()

In [2]:
x = torch.randn(8, 3, 32, 48)   # [B, C, H, W] = [8, 3, 32, 48]
x_flat = x.flatten(start_dim=1)  # [8, 4608]
print("flatten:", x_flat.shape)

x_view = x.view(x.size(0), -1)   # [8, 4608]
print("view:", x_view.shape)

# reshape to same shape trying to avoid copy if possible
x_reshape = x.reshape(8, -1)     # [8, 4608]
print("reshape:", x_reshape.shape)

flatten: torch.Size([8, 4608])
view: torch.Size([8, 4608])
reshape: torch.Size([8, 4608])


In [3]:
v = torch.randn(8, 1)    # [B, 1]

v2 = v.unsqueeze(dim=2)      # [8, 1, 1]
print("unsqueeze:", v2.shape)

v3 = v2.squeeze(dim=1)       # [8, 1]
print("squeeze dim1:", v3.shape)

v4 = v2.squeeze()        # 删除所有长度为1的轴 -> [8]
print("squeeze all:", v4.shape)

unsqueeze: torch.Size([8, 1, 1])
squeeze dim1: torch.Size([8, 1])
squeeze all: torch.Size([8])


In [4]:
# [B, C, H, W] -> [B, H, W, C]
y1 = x.permute(0, 2, 3, 1)
print("permute:", y1.shape)

y2 = torch.movedim(x, 1, -1)     # same as permute
print("movedim:", y2.shape)

y3 = x.transpose(2, 3)           # [8, 3, 32, 48] -> [8, 3, 48, 32]
print("transpose:", y3.shape)

# 典型 CNN flatten 到 linear 的写法：
# 非连续: permute 打乱 memory layout
y = x.permute(0, 2, 3, 1)        # [8, 32, 48, 3]

# 需要 contiguous 让内存重新排布
y = y.contiguous()

# 再 flatten
y = y.view(8, -1)                # [8, 4608]
print("combo:", y.shape)

permute: torch.Size([8, 32, 48, 3])
movedim: torch.Size([8, 32, 48, 3])
transpose: torch.Size([8, 3, 48, 32])
combo: torch.Size([8, 4608])


## Broadcasting

规则：从右往左对齐维度；维度相等或其中一个为 1 才能广播。
- x[..., None]：在最后插 1 维

- x[None, ...]：在最前插 1 维

- x.expand(...)：不拷贝，只是“看起来变大”（stride trick）

- x.repeat(...)：真拷贝（更耗内存）

看到 None/unsqueeze + expand → 99% 是在做 broadcast 对齐维度。

expand is cheap but requires the dimension to be 1 (or added via unsqueeze). If you need real copies, use repeat.

- broadcast rules
    - 从最右侧维度开始向左比较。

    - 两个维度兼容的条件：要么相等，要么其中一个为 1，要么某个张量缺少该维（视为前面补 1）。

    - 结果维度取每个位置上的较大值，维度为 1 的那一方会被“拉伸”（不复制数据）。

    - 如果某一维既不相等又不为 1，则无法广播，报错。

    - 例子：

a: (2, 1, 4)

b: (1, 3, 4)
从右往左比较：4 vs 4（ok），1 vs 3（1 可广播），2 vs 1（1 可广播），所以结果形状是 (2, 3, 4)。


In [5]:
x = torch.tensor([1, 2, 3])          # shape (3,)
y = torch.tensor([10, 20, 30])       # shape (3,)

# 1) x[..., None]: insert a trailing dim
x_col = x[..., None]                 # shape (3,1)
# 2) x[None, ...]: insert a leading dim
x_row = x[None, ...]                 # shape (1,3)

# Broadcast to outer sum via None/unsqueeze
outer = x_col + y                    # shapes (3,1) + (3,) -> (3,3)
print("outer via None:\n", outer)
print(f"outer shape: {outer.shape}")

# 3) expand: no copy, just view with stride trick
x2 = torch.tensor([1, 2, 3])          # shape (3,)
# Make a leading singleton dim, then expand to 4 rows
x2_expanded = x2.unsqueeze(0).expand(4, -1)  # shape (4,3), no data copy
print("expanded (no copy):\n", x2_expanded)
print(f"expanded shape: {x2_expanded.shape}")

# Expand a column vector to a full matrix
col = torch.tensor([[10], [20]])     # shape (2,1)
mat = col.expand(-1, 5)              # shape (2,5)
print(f"expanded shape: {mat.shape}")

# 4) repeat: real copy
x2_repeated = x2.unsqueeze(0).repeat(4, 1)   # shape (4,3), data copied
print("repeated (copy):\n", x2_repeated)
print(f"repeated shape: {x2_repeated.shape}")

# Broadcasting with mismatched dims
a = torch.randn(2, 1, 4)   # shape (2,1,4)
b = torch.randn(1, 3, 4)   # shape (1,3,4)
c = a + b                  # broadcast to (2,3,4)
print("a+b shape:", c.shape)

outer via None:
 tensor([[11, 21, 31],
        [12, 22, 32],
        [13, 23, 33]])
outer shape: torch.Size([3, 3])
expanded (no copy):
 tensor([[1, 2, 3],
        [1, 2, 3],
        [1, 2, 3],
        [1, 2, 3]])
expanded shape: torch.Size([4, 3])
expanded shape: torch.Size([2, 5])
repeated (copy):
 tensor([[1, 2, 3],
        [1, 2, 3],
        [1, 2, 3],
        [1, 2, 3]])
repeated shape: torch.Size([4, 3])
a+b shape: torch.Size([2, 3, 4])


## Grid generation（meshgrid：坐标网格/位置）

- yy, xx = torch.meshgrid(ys, xs, indexing="ij")， yy.shape == xx.shape == [H, W]

- grid = torch.stack([xx, yy], dim=-1) → [H, W, 2]

- grid.reshape(-1, 2) → 点集 [H*W, 2]

看到 meshgrid/stack → 在造坐标、位置编码、采样网格。

In [6]:
import torch

# coordinate ranges
xs = torch.linspace(-1.0, 1.0, steps=4)  # W = 4
ys = torch.linspace(-2.0, 2.0, steps=3)  # H = 3

# meshgrid with ij indexing (row = y, col = x)
yy, xx = torch.meshgrid(ys, xs, indexing="ij")
print("xx shape:", xx.shape)  # (H, W)
print("yy shape:", yy.shape)  # (H, W)

# stack into a grid of (x,y) pairs: [H, W, 2]
# dim=-1 表示在最后一个维度上堆叠
grid = torch.stack([xx, yy], dim=-1)
print("grid shape:", grid.shape)
print("grid:\n", grid)

# flatten to point set: [H*W, 2]
points = grid.reshape(-1, 2)
print("points shape:", points.shape)
# print("points:\n", points)

xx shape: torch.Size([3, 4])
yy shape: torch.Size([3, 4])
grid shape: torch.Size([3, 4, 2])
grid:
 tensor([[[-1.0000, -2.0000],
         [-0.3333, -2.0000],
         [ 0.3333, -2.0000],
         [ 1.0000, -2.0000]],

        [[-1.0000,  0.0000],
         [-0.3333,  0.0000],
         [ 0.3333,  0.0000],
         [ 1.0000,  0.0000]],

        [[-1.0000,  2.0000],
         [-0.3333,  2.0000],
         [ 0.3333,  2.0000],
         [ 1.0000,  2.0000]]])
points shape: torch.Size([12, 2])


## Gather / Scatter（索引驱动的取值/写回）
### gather：按 index “取值”
- 沿着某个维度 dim，用 index 给出的坐标，把 input 里的元素“捞”出来；输出的形状由 index 决定。
- index 必须是 LongTensor（int64）。
- index 和 x 维数相同（rank 相同）。
- index 在非 dim 的各维大小要能和输出对齐（需要expand）。
- index 的取值必须在 [0, x.size(dim)-1]。

例子 tensor =(B, N, C), B=2, N=5, C=3 在 dim=1（N维度） 取K（K=2）个 （每次取都会把一整条C维向量取出来，C=c0, c01, c2）
取法 index=[[0, 2], [4, 1]] 也就是说对
- batch0 选 n=0 和 n=2
- batch1 选 n=4 和 n=1

index实际上是（B，K） k代表N维上的坐标。

实际取值如下图 batch=0 为例，输出的shape = (B， K， C) = (2, 2, 3)


![gather](./image/gather_indices.png)

实际做的就是`y[b, k, :] = x[b, idx[b, k], :]`

### scatter：按 index “写回/覆盖”

out.scatter_(dim, index, src)：把 src 写到 out 指定位置（覆盖）

out.scatter_add_(dim, index, src)：写回时做累加（处理重复 index 更自然）

直觉

- gather：从大张量按索引“捞出来”

- scatter：把小张量按索引“塞回去”

- scatter_add：塞回去还要“加账”

In [7]:
import torch

# gather
# x: [B, N, C]  -> 在 dim=1 按 index 取值
B, N, C = 2, 5, 3
x = torch.arange(B*N*C).reshape(B, N, C)
# x.shape = (2, 5, 3)

# idx: [B, K]，想在 token 维 (dim=1) 选 K 个
K = 2
idx = torch.tensor([[0, 2],
                    [4, 1]])  # shape (2,2)
# 表达的是
# batch0：k=0 选 n=0（对 c=0,1,2 都一样），k=1 选 n=2（对 c=0,1,2 都一样）
# batch1：k=0 选 n=4（对 c=0,1,2 都一样），k=1 选 n=1（对 c=0,1,2 都一样）

# 先扩成和 x 同步的形状，在 dim=1 上用 gather
idx_exp = idx.unsqueeze(-1).expand(B, K, C)  # shape  (2,2,1) --> (2,2,3)
y = x.gather(dim=1, index=idx_exp)           # y.shape = (2,2,3)
print("x shape:", x.shape)
print("idx shape:", idx.shape)
print(f"idx.unsqueeze(-1) shape:", idx.unsqueeze(-1).shape)
print("idx_exp shape:", idx_exp.shape)
print(f"idx_exp values:\n{idx_exp}")
print("y shape:", y.shape)
print("y:\n", y)

# 说明：结果 y 的形状等于 index 的形状 (2,2,3)，其中 dim=1 的长度来自 idx 的 K=2，
# 其他维度 (B, C) 与原 x 对齐。

# scatter
# 将 src 写回 out 的指定位置（覆盖）
out = torch.zeros_like(x)  # shape (2,5,3)
src = torch.ones_like(y)   # 形状要与 index 对齐 (2,2,3)
out.scatter_(dim=1, index=idx_exp, src=src)
print("scatter result (cover):\n", out)

# 若 index 有重复，覆盖可能丢信息，可以用 scatter_add_ 做累加
out2 = torch.zeros_like(x)
src2 = torch.full_like(y, 2.0)
out2.scatter_add_(dim=1, index=idx_exp, src=src2)
print("scatter_add result (add on duplicates):\n", out2)

x shape: torch.Size([2, 5, 3])
idx shape: torch.Size([2, 2])
idx.unsqueeze(-1) shape: torch.Size([2, 2, 1])
idx_exp shape: torch.Size([2, 2, 3])
idx_exp values:
tensor([[[0, 0, 0],
         [2, 2, 2]],

        [[4, 4, 4],
         [1, 1, 1]]])
y shape: torch.Size([2, 2, 3])
y:
 tensor([[[ 0,  1,  2],
         [ 6,  7,  8]],

        [[27, 28, 29],
         [18, 19, 20]]])
scatter result (cover):
 tensor([[[1, 1, 1],
         [0, 0, 0],
         [1, 1, 1],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [1, 1, 1],
         [0, 0, 0],
         [0, 0, 0],
         [1, 1, 1]]])
scatter_add result (add on duplicates):
 tensor([[[2, 2, 2],
         [0, 0, 0],
         [2, 2, 2],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [2, 2, 2],
         [0, 0, 0],
         [0, 0, 0],
         [2, 2, 2]]])


idx_exp[b, k, c] 表示：为了得到 y[b,k,c]，沿 dim=1 要去取哪个 n。
```text
tensor([
  [[0, 0, 0],
   [2, 2, 2]],

  [[4, 4, 4],
   [1, 1, 1]]
])
```
- batch 0

    - idx_exp[0,0,:] = [0,0,0]

    意思是：当 b=0,k=0 时，不管 c=0/1/2，都取 n=0

    所以：

    y[0,0,0] = x[0,0,0]

    y[0,0,1] = x[0,0,1]

    y[0,0,2] = x[0,0,2]

    - idx_exp[0,1,:] = [2,2,2]

    意思是：当 b=0,k=1 时，所有 channel 都取 n=2

    所以 y[0,1,:] = x[0,2,:]

- batch 1

    - idx_exp[1,0,:] = [4,4,4] → y[1,0,:] = x[1,4,:]

    - idx_exp[1,1,:] = [1,1,1] → y[1,1,:] = x[1,1,:]

## Reduction / Selection（常和 gather/scatter 一起出现）

- topk：values, indices = x.topk(k, dim=...)

- argmax / max：vals, idx = x.max(dim=...)

- sum/mean/min/max：降维聚合（注意 keepdim=True 是否保维）

常见链路：

topk -> indices -> gather（取对应向量/特征）

indices -> scatter/scatter_add（写回 one-hot / routing 权重）

In [10]:
import torch

torch.manual_seed(1)

batch, items, feat_dim = 2, 5, 3
scores = torch.randn(batch, items)            # e.g., logits per item
features = torch.randn(batch, items, feat_dim)  # feature vector per item

# topk -> indices -> gather (pick top-k features)
k = 2
# within each batch row, pick the top‑k items
top_vals, top_idx = scores.topk(k, dim=1)
# selects along the items dimension using those indices, pulling out the feature vectors for the top‑k items.
# top_idx.unsqueeze(-1) → [batch, k, 1], so each index has a slot for the feature dimension.
# .expand(-1, -1, feat_dim) → [batch, k, feat_dim], replicating each index across the feature dimension (no extra memory).
# picked_features has shape [batch, k, feat_dim] containing the feature vectors corresponding to the top‑k items for each batch row.
picked_features = features.gather(
    1, top_idx.unsqueeze(-1).expand(-1, -1, feat_dim)
)

# argmax / max (with and without keepdim)
max_vals, max_idx = scores.max(dim=1)              # shape: [batch]
max_vals_kd, max_idx_kd = scores.max(dim=1, keepdim=True)  # shape: [batch, 1]

# indices -> scatter (one-hot) and scatter_add (accumulate weights)
one_hot = torch.zeros_like(scores)
one_hot.scatter_(1, top_idx, 1.0)  # write ones at top-k positions

weights = torch.rand(batch, k)     # routing weights for each top-k index
routing = torch.zeros_like(scores)
routing.scatter_add_(1, top_idx, weights)  # add weights back to original slots

print("scores:\n", scores)
print("top_vals:\n", top_vals)
print("top_idx:\n", top_idx)
print("picked_features.shape:", picked_features.shape)
print("max_vals:", max_vals)
print("max_idx:", max_idx)
print("max_vals_kd (keepdim):\n", max_vals_kd)
print("one_hot:\n", one_hot)
print("routing (after scatter_add):\n", routing)

scores:
 tensor([[ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519],
        [-0.1661, -1.5228,  0.3817, -1.0276, -0.5631]])
top_vals:
 tensor([[ 0.6614,  0.6213],
        [ 0.3817, -0.1661]])
top_idx:
 tensor([[0, 3],
        [2, 0]])
picked_features.shape: torch.Size([2, 2, 3])
max_vals: tensor([0.6614, 0.3817])
max_idx: tensor([0, 2])
max_vals_kd (keepdim):
 tensor([[0.6614],
        [0.3817]])
one_hot:
 tensor([[1., 0., 0., 1., 0.],
        [1., 0., 1., 0., 0.]])
routing (after scatter_add):
 tensor([[0.4903, 0.0000, 0.0000, 0.5730, 0.0000],
        [0.1452, 0.0000, 0.1205, 0.0000, 0.0000]])


## 总结： 三种"按 index 取值"的方式：高级索引 vs gather vs take_along_dim

### 场景回顾
给定 `x: [B, N, C]`，想按某个维度（如 dim=1）用 index `idx: [B, K]` 选出 K 个行，得到 `y: [B, K, C]`。

---

### A) 高级索引（Advanced Indexing）—— 最直观

**核心思想**：就像 NumPy 一样直接"选行"。

```python
b = torch.arange(B)[:, None]  # [B, 1]
y = x[b, idx]                 # idx:[B,K] -> y:[B,K,C]
```

**公式逻辑**：
$$y[b,k,:] = x[b, \text{idx}[b,k], :]$$

**优点**
- 最像"选行"，最容易读懂
- 不需要 `unsqueeze + expand`，代码最简洁

**缺点**
- 高级索引的 shape 规则比较复杂（混用 slice + 整型索引张量时很绕）
- 在某些编译/导出场景（`torch.compile` / ONNX）可控性/可预测性不如 `gather`
- 维度重排规则隐晦，容易踩坑

---

### B) gather(dim=...) —— 最"通用"、最规则

**核心思想**：严格按 index 张量的形状作为输出形状，逐元素对应取值。

```python
idx_exp = idx.unsqueeze(-1).expand(B, K, C)  # [B,K] -> [B,K,1] -> [B,K,C]
y = x.gather(1, idx_exp)                      # [B,K,C]
```

**公式逻辑**：
$$y[b,k,c] = x[b, \text{index}[b,k,c], c]$$

因此必须构造 `index[b,k,c] \equiv \text{idx}[b,k]`（对所有 c 重复）。

**优点**
- 规则非常硬：**输出 shape = index shape**
- 在多维、批量、图编译里更稳定（形状推理清晰）
- 不会触发高级索引那套"维度重排"规则
- 和 `scatter` / `scatter_add` 天然配对（MoE routing、top-k 后操作）

**缺点**
- 反直觉：需要把 index 做到跟输出同 rank（必须 `unsqueeze + expand`）
- 容易在 expand 步骤上迷糊

**为什么要 expand？**
> `gather` 是逐元素操作，每个输出元素 `y[b,k,c]` 都需要一个对应的 index 位置。
> 虽然逻辑上我们只想在 `dim=1` 上索引（用 `idx[b,k]`），但 `gather` 要求 index 和输出同 rank。
> 所以必须"复制" `idx[b,k]` 到 C 维：`idx.unsqueeze(-1).expand(-1, -1, C)`。

---

### C) torch.take_along_dim —— "更像 numpy.take_along_axis"

**核心思想**：API 和 `gather` 相近，但语义更像 NumPy。

```python
idx_exp = idx.unsqueeze(-1).expand(B, K, C)  # 同样需要 expand
y = torch.take_along_dim(x, idx_exp, dim=1)   # [B,K,C]
```

**优点**
- 语义好读：沿某个 dim "take along"
- 规则接近 `gather`（输出 shape ~ index shape）
- 从 NumPy 迁移的人会更熟悉

**缺点**
- 同样需要把 index 变到和输出同 rank（`[B,K]` → `[B,K,C]`）
- 本质和 `gather` 差不多，选型意义不大

---

### 一句话选型指南（最实用）

| 场景 | 推荐 | 理由 |
|------|------|------|
| 快速验证、想最直观 | **高级索引** | 代码最简洁，最像 NumPy |
| 通用模块、形状要稳定 | **gather** | 规则硬、可预测、容易编译导出 |
| 和 `scatter` / `scatter_add` 配对 | **gather** | 天然搭配，形状对齐 |
| 从 NumPy 迁移、用过 `take_along_axis` | **take_along_dim** | 心智模型一致 |
| MoE routing、top-k 后写回 | **gather + scatter** | 这是行业标准 pipeline |

---

### 什么时候 gather 比高级索引更"安全"

1. **不想触发复杂 shape 行为**  
   混合多个索引张量时，高级索引会自动广播和重排维度，容易出现意想不到的输出 shape。

2. **需要"index 张量决定输出形状"的强规则**  
   例如 MoE 的 routing、softmax 后的 topk 选择。高级索引在这里容易出错。

3. **后面要配套 scatter / scatter_add 写回**  
   ```python
   # gather 取出来
   idx = top_indices  # [B, K]
   idx_exp = idx.unsqueeze(-1).expand(B, K, C)
   picked = x.gather(1, idx_exp)  # [B, K, C]
   
   # scatter 写回（形状自动对齐）
   out = torch.zeros_like(x)
   out.scatter_(1, idx_exp, picked)  # [B, N, C]
   ```
   这套组合在库代码（PyTorch 自己的 MoE、注意力机制、路由模块）里是标配。

4. **涉及 torch.compile / ONNX 导出**  
   `gather` 的形状推理清晰、在各平台支持度高；高级索引在某些编译器上可能有意想不到的行为。

---

### 完整对比表

| 特性 | 高级索引 | gather | take_along_dim |
|------|---------|--------|----------------|
| 代码简洁度 | ★★★ | ★ | ★★ |
| 易读性 | ★★★ | ★★ | ★★★ |
| 形状规则清晰 | ★ | ★★★ | ★★★ |
| 编译导出友好 | ★ | ★★★ | ★★★ |
| 和 scatter 配对 | ▲ | ★★★ | ★★★ |
| 库代码常见度 | ★★ | ★★★ | ★ |